In [1]:
import pandas as pd
import numpy as np

import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import FunctionTransformer

import nltk
nltk.download('wordnet')

from nltk import tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from nltk import PorterStemmer

from time import time

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/cindywong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from sklearn.pipeline import Pipeline

In [3]:
# 2736 rows of dataset
df = pd.read_csv('data/ufo.csv')
df.head()

,Datetime,Location,Shape,Duration,Text,Notes
0,5/6/2017 05:00,"Camp McGregor, NM",Light,10 minute,Light seen over mountain's east of Camp McGre...,Report appears to us to be consistent with t...
1,5/5/2017 11:30,"Austin, TX",Disk,3 second,"Flying saucer descends, possibly lands in Nor...",We would like to communicate with this witne...
2,5/4/2017 21:27,"Phoenix, AZ",Circle,15 second,Orange round sphere. Orange glowing sphere f...,"We have amended the time above, to reflect a..."
3,5/4/2017 18:30,"Phoenix, AZ",Teardrop,5 minute,Flying corkscrews Looking to th east at abou...,Source of the report elects to remain anonymous
4,5/4/2017 04:50,"Taft, CA",Changing,20 second,I'm a truck driver and I've seen the reddish/...,"Witness indicates ""Taft, Indiana"" in origina..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2736 entries, 0 to 2735
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Datetime  2736 non-null   object
 1   Location  2736 non-null   object
 2   Shape     2736 non-null   object
 3   Duration  2736 non-null   object
 4   Text      2736 non-null   object
 5   Notes     2736 non-null   object
dtypes: object(6)
memory usage: 128.4+ KB


In [5]:
df['Text']

0        Light seen over mountain's east of Camp McGre...
1        Flying saucer descends, possibly lands in Nor...
2        Orange round sphere.  Orange glowing sphere f...
3        Flying corkscrews  Looking to th east at abou...
4        I'm a truck driver and I've seen the reddish/...
                              ...                        
2731     Brightly lit craft flew and hovered right in ...
2732     Triangle bright flashing white light, releasi...
2733     Light, orange, red, fast speed.  My gf and I ...
2734     Three circular, flashing UFO's moving in erra...
2735     Fireball came in from east and made sharp tur...
Name: Text, Length: 2736, dtype: object

In [6]:
# for idx, doc in enumerate(text_pre):
#     #df.iloc[:, 5].str.replace('\d+', '') # for digits
#     df.iloc[idx, 4] = doc.lower()

In [7]:
df['Text'] = df['Text'].str.replace('\d+', '') # for digits
df['Text'] = df['Text'].str.replace(r'(\b\w{1,2}\b)', '') # for words
df['Text'] = df['Text'].str.replace(r'[^\w\s]+', '') # for punctuation
df['Text'] = df['Text'].str.lower()

In [8]:
df['Text']

0        light seen over mountain east  camp mcgregor ...
1        flying saucer descends possibly lands  north ...
2        orange round sphere  orange glowing sphere fl...
3        flying corkscrews  looking   east  about   sa...
4          truck driver and  seen the reddishorange ba...
                              ...                        
2731     brightly lit craft flew and hovered right  fr...
2732     triangle bright flashing white light releasin...
2733     light orange red fast speed    and  were look...
2734     three circular flashing ufo moving  erratic p...
2735     fireball came  from east and made sharp turn ...
Name: Text, Length: 2736, dtype: object

 [pipelinize source](https://evisionindia.wordpress.com/2020/03/06/setting-up-text-preprocessing-pipeline-using-scikit-learn-and-spacy-learn-how-to-tokenize-lemmatize-remove-stop-words-and-punctuation-with-sklearn-pipelines/)

In [9]:
# from string import *

# class PorterTokenizer:
#      def __init__(self):
            
#             self.ps = PorterStemmer()
#             self.lower      = lower
#             self.strip      = strip
#             self.stopwords  = stopwords or set(sw.words('english'))
#             self.punct      = punct or set(string.punctuation)
#             self.lemmatizer = WordNetLemmatizer()
#      def __call__(self, doc):
#          return [self.ps.stem(t) for t in word_tokenize(doc)]

In [10]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [11]:
# from nltk import word_tokenize          
# from nltk.stem import WordNetLemmatizer 
# class LemmaTokenizer(object):
#     def __init__(self):
#         self.wnl = WordNetLemmatizer()
#     def __call__(self, articles):
#         return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

# tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(),
#                                 strip_accents = 'unicode', # works 
#                                 stop_words = 'english', # works
#                                 lowercase = True, # works
#                                 max_df = 0.5, # works
#                                 min_df = 10) # works

In [12]:
from sklearn.decomposition import NMF

In [13]:
n_components = 10
n_features=10000

In [14]:
vect = TfidfVectorizer(stop_words='english', max_features=n_features,
                       ngram_range=(1,2),
                       tokenizer=LemmaTokenizer(),
                       strip_accents = 'unicode', # works 
                       lowercase = True, # works
                       max_df = 0.5, # works
                       min_df = 10 # works
                       #, tokenizer=PorterTokenizer()
                      )

In [16]:
words = vect.fit_transform(df['Text'])
V = words.toarray()
V.shape

(2736, 3098)

In [17]:
nmf = NMF(n_components=n_components)
nmf.fit(V)
W = nmf.transform(V)
H = nmf.components_

nmf.reconstruction_err_

49.46496217148482

In [22]:
H.shape

(10, 3098)

In [23]:
feature_names = vect.get_feature_names()

In [24]:
feature_names[:5]

['able', 'abruptly', 'absolutely', 'absolutely sound', 'accelerated']

In [25]:
index_val = np.argsort(H)[:, -1:-11:-1]

for i, lat_feat in enumerate(index_val):
    print('%d: %s'%(i+1,', '.join([feature_names[n] for n in lat_feat])))

1: like, looked, just, saw, star, time, went, look, looked like, night
2: provides, information, provides contact, contact information, contact, anonymous provides, witness elect, bright, witness provides, disappeared
3: launch, missile, missile launch, note navy, navy missile, navy, blue, bright, cloud, white
4: minute, minute later, minute nuforc, later, sky minute, watched, stationary, minute moved, stayed, hovered
5: object, object wa, appeared, shaped object, photo, video, flying, shaped, object appeared, high
6: source, source report, report, note source, report elect, remain anonymous, report indicates, date approximate, note information, information provided
7: craft, red, white, white light, red light, flashing, triangle, low, flying, bright white
8: orange, orange light, east, moving, west, fireball, bright orange, orb, north, south
9: second, flash, second nuforc, green, second later, later, meteor, disappeared, second wa, second disappeared
10: date, approximate, indicates,

In [26]:
print("Performing dimensionality reduction using LSA")
t0 = time()
# Vectorizer results are normalized, which makes KMeans behave as
# spherical k-means for better results. Since LSA/SVD results are
# not normalized, we have to redo the normalization.
svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(words)

print("done in %fs" % (time() - t0))

explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
    int(explained_variance * 100)))

print()

Performing dimensionality reduction using LSA
done in 0.584951s
Explained variance of the SVD step: 6%



In [30]:
km = KMeans(n_clusters=10, init='k-means++', max_iter=100, n_init=1,
            verbose=False)
print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
# print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
# print("Adjusted Rand-Index: %.3f"
#       % metrics.adjusted_rand_score(labels, km.labels_))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, km.labels_, sample_size=1000))

# print()

Clustering sparse data with KMeans(max_iter=100, n_clusters=10, n_init=1, verbose=False)
done in 0.171s



In [28]:
km.cluster_centers_

array([[ 0.67460592,  0.17790962,  0.01296322,  0.03217064,  0.04473398,
        -0.09893726,  0.38932031,  0.26785883,  0.10183356,  0.04491865],
       [ 0.69603992,  0.0181176 , -0.08183118, -0.00890915, -0.06289363,
        -0.04708611, -0.31576178,  0.35118151,  0.07361176, -0.1112587 ],
       [ 0.74655288,  0.18387593, -0.00842903, -0.00325309,  0.3906447 ,
         0.11267978, -0.06311554, -0.0595771 , -0.07664136,  0.11868286],
       [ 0.67019299,  0.06136659,  0.1790519 , -0.06855207, -0.02738174,
        -0.18477233, -0.04170666, -0.19724955,  0.34532481, -0.0252963 ],
       [ 0.46940764, -0.2904576 ,  0.60879164, -0.11577357, -0.05811764,
         0.09781153,  0.03390593,  0.05367022, -0.08212395, -0.00722883],
       [ 0.6487843 , -0.362718  , -0.29556903,  0.01278014, -0.21750306,
         0.04304135,  0.00510381, -0.01572252,  0.08924045,  0.26576682],
       [ 0.56895269,  0.13122338, -0.14753645,  0.03675873, -0.26106528,
         0.46966032,  0.09891121, -0.14555687

In [33]:
print("Top terms per cluster:")
original_space_centroids = svd.inverse_transform(km.cluster_centers_)
order_centroids = original_space_centroids.argsort()[:, ::-1]
#     else:
#         order_centroids = km.cluster_centers_.argsort()[:, ::-1]

terms = vect.get_feature_names()
for i in range(1, n_components):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :5]:
        print(' %s' % terms[ind], end='')
    print()

Top terms per cluster:
Cluster 1: object object wa appeared moving time
Cluster 2: provides information provides contact contact information witness elect
Cluster 3: launch missile missile launch note navy navy missile
Cluster 4: minute like star bright looked
Cluster 5: second bright white green red
Cluster 6: like saw just looked object
Cluster 7: date approximate indicates indicates date sighting
Cluster 8: source report source report note source report elect
Cluster 9: orange moving witness elect orange light second
